In [21]:
import requests
import json
import pandas as pd

import plotly.express as px

In [22]:

argdict = {
    'state': 'NY',
    'climate_zone': '5A',
    'projection_case': 'MidCase',
    'design_areas': [
        {
            'type': 'ApartmentHighRise',
            'area': 200000,
            'heating_fuel': 'electricity',
            'dhw_fuel': 'Electricity',
            'heating_cop': 2.5,
            'dhw_cop': 2.5,
            'ashrae_standard': 'STD2013'
        },
        {
            'type': 'OfficeSmall',
            'area': 10000,
            'heating_fuel': 'Natural Gas',
            'dhw_fuel': 'Natural Gas',
            'heating_cop': 0.8,
            'dhw_cop': 0.8,
            'ashrae_standard': 'STD2013'
        }],
}

payload = json.dumps(argdict)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'


r = requests.get(
    f'{localhost}/get_projection_from_reference_buildings/?params={payload}'
)
js = r.json()

js

{'emissions_projection': [{'year': 2018.0,
   'elec_kg_per_kbtu': 0.05254982415005862,
   'kg_co2_absolute': 389573.1973904364,
   'kg_co2_per_sf': 1.855110463763983},
  {'year': 2020.0,
   'elec_kg_per_kbtu': 0.05046893317702227,
   'kg_co2_absolute': 374417.98533580895,
   'kg_co2_per_sf': 1.7829427873133759},
  {'year': 2022.0,
   'elec_kg_per_kbtu': 0.05451348182883939,
   'kg_co2_absolute': 403874.5946814228,
   'kg_co2_per_sf': 1.923212355625823},
  {'year': 2024.0,
   'elec_kg_per_kbtu': 0.03930246189917937,
   'kg_co2_absolute': 293092.12909900537,
   'kg_co2_per_sf': 1.3956768052333588},
  {'year': 2026.0,
   'elec_kg_per_kbtu': 0.0335873388042204,
   'kg_co2_absolute': 251468.6593715075,
   'kg_co2_per_sf': 1.1974698065309881},
  {'year': 2028.0,
   'elec_kg_per_kbtu': 0.02816529894490035,
   'kg_co2_absolute': 211979.72655311204,
   'kg_co2_per_sf': 1.0094272693005335},
  {'year': 2030.0,
   'elec_kg_per_kbtu': 0.024882766705744432,
   'kg_co2_absolute': 188072.91317116455,


In [23]:
# manual enduses

config = {
    'state': 'MA',
    'projection_case': 'MidCase',
    'area': 215000,
    'enduses': [
        {
            'enduse': 'elec',
            'fuel': 'electricity',
            'kbtu': 7129e3
        },
        {
            'enduse': 'gas',
            'fuel': 'natural gas',
            'kbtu': 3111e3
        },
    ]
}

payload = json.dumps(config)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'

r = requests.get(
    f'{localhost}/get_projection_from_manual_enduses/?params={payload}'
)

js = r.json()





In [24]:
df_projection = pd.DataFrame(js['emissions_projection'])
df_enduses = pd.DataFrame(js['enduses']['enduses_per_sf'])
df_enduses
px.pie(df_enduses, values='kbtu', names='fuel', labels='kbtu', width=250, height=250, hole=0.5, title='site energy')
# px.line(df_projection, x='year', y='kg_co2_per_sf')


